In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tqdm

In [52]:
df = pd.read_csv("QUANTT_dataset.csv").iloc[:,1:]

In [53]:
df.head()

,Date,Stock price,Strike price,T,Risk-Free Rate,Volatility
0,2014-01-02,9.9867,7.480840,0.603493,0.0300,0.5527
1,2014-01-03,10.0000,19.014286,0.357814,0.0301,0.5527
2,2014-01-06,10.0000,14.639879,0.647817,0.0298,0.5527
3,2014-01-07,9.8413,11.783155,0.122921,0.0296,0.5527
4,2014-01-08,9.9233,3.096440,0.888659,0.0301,0.5527


In [33]:
# Generating random values for Strike price and T
np.random.seed(42)  # Setting a seed for reproducibility
df["Strike price"] = np.random.uniform(0, 2*df["Stock price"], len(df))
df["T"] = np.random.uniform(0, 1, size=len(df))

In [63]:
def black_scholes(S, X, T, r, sigma):
    """
    S: Current stock price
    X: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (expressed as a decimal)
    sigma: Volatility of the stock's return
    """
    # Calculate d1 and d2
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
   
    # Calculate the call option price
    call_price = (S * norm.cdf(d1)) - (X * np.exp(-r * T) * norm.cdf(d2))
   
    # Calculate the put option price
    put_price = (X * np.exp(-r * T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
   
    return call_price

def monte_carlo(Stock_P, Strike_P, vol, r, T, N=10, M=1_000):
    # Constants. These apply to the formula -----> delta(x) = u*delta(t) + sigma*delta(z)
    dt = T/N                                #   timestep. I.E., if T is 5 years and N is 10, this will give each time step as 0.5 years
    u_dt = (r-0.5*vol**2)*dt                #   drift term (u)  
    vol_sqrt_dt = vol*np.sqrt(dt)           #   From equation on line 43, delta(z) becomes sqrt(delta(t)) when time step is applied
                                            #   -----> StockPriceNow = StockPricePrevious*e^(u*delta(t) + sigma*delta(z))
    # Standard error placeholders
    totalCT = 0
    totalCT2 = 0

    # MONTE CARLO METHOD
    for i in range(M):  #M is total simulations
        lnStock_P = np.log(Stock_P)
        for j in range(N):  #N is total timesteps
            lnStock_P = lnStock_P + u_dt + vol_sqrt_dt*np.random.normal()

        ST = np.exp(lnStock_P) # e^ln cancels out and leaves the stock price as ST
        CT = max(0, ST - Strike_P) 
        totalCT = totalCT + CT
        totalCT2 = totalCT2 +CT*CT

    # Find the call value and the SE
    C0 = np.exp(-r*T)*totalCT/M    # C0 is call value. Comes from formula ------> C0 = (1/M)*(SIGMA(  M(top of sigma)...i=1(bottom of sigma) C0 ))
    sigma = np.sqrt((totalCT2 - totalCT*totalCT/M)*np.exp(-2*r*T) / (M-1)) # for standard error. Has nothing to do with uppercase SIGMA from line 65
    SE = sigma/np.sqrt(M) #find the standard error. This is essentially based on number of sims and call value at each time point

    return C0

def binomial_tree(S0, K, T, r, sigma, N = 100):
    
    # Calculations
    dt = T / N  # length of each step
    u = np.exp(sigma * np.sqrt(dt))  # up-factor
    d = 1 / u  # down-factor
    p = (np.exp(r * dt) - d) / (u - d)  # risk-neutral probability

    # Initialize the Stock Price Tree
    stock_price_tree = np.zeros((N + 1, N + 1))
    stock_price_tree[0, 0] = S0
    for i in range(1, N + 1):
        stock_price_tree[i, 0] = stock_price_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_price_tree[i, j] = stock_price_tree[i - 1, j - 1] * d

    # Initialize the Option Value Tree for a Call Option
    option_value_tree = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        option_value_tree[N, j] = max(0, stock_price_tree[N, j] - K)

    # Perform Backward Induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_value_tree[i, j] = np.exp(-r * dt) * (p * option_value_tree[i + 1, j] + (1 - p) * option_value_tree[i + 1, j + 1])

    # Result
    option_price = option_value_tree[0, 0]

    return option_price

In [61]:
# Setting up a progress bar for the DataFrame's length
for i in tqdm.tqdm(df.index, desc='Calculating Values', position=0, leave=True):
    row = df.loc[i]
    df.at[i, 'BS Value'] = black_scholes(row['Stock price'], row['Strike price'], row['T'], row['Risk-Free Rate'], row['Volatility'])
    df.at[i, 'MC Value'] = monte_carlo(row['Stock price'], row['Strike price'], row['Volatility'], row['Risk-Free Rate'], row['T'])
    df.at[i, 'BT Value'] = binomial_tree(row['Stock price'], row['Strike price'], row['T'], row['Risk-Free Rate'], row['Volatility'])

Calculating Values:   0%|          | 0/2577 [00:00<?, ?it/s]

Calculating Values: 100%|██████████| 2577/2577 [07:50<00:00,  5.48it/s]


In [64]:
# Setting up a progress bar for the DataFrame's length
for i in tqdm.tqdm(df.index, desc='Calculating Values', position=0, leave=True):
    row = df.loc[i]
    df.at[i, 'BT Value'] = binomial_tree(row['Stock price'], row['Strike price'], row['T'], row['Risk-Free Rate'], row['Volatility'])

Calculating Values: 100%|██████████| 2577/2577 [00:39<00:00, 65.64it/s]


In [65]:
df['Average Value'] = df[['BT Value', 'BS Value', 'MC Value']].mean(axis=1)

In [66]:
df.head()

,Date,Stock price,Strike price,T,Risk-Free Rate,Volatility,BS Value,MC Value,BT Value,Average Value
0,2014-01-02,9.9867,7.480840,0.603493,0.0300,0.5527,3.145632,3.164122,3.147523,3.152425
1,2014-01-03,10.0000,19.014286,0.357814,0.0301,0.5527,0.048183,0.048735,0.047745,0.048221
2,2014-01-06,10.0000,14.639879,0.647817,0.0298,0.5527,0.616661,0.658655,0.617788,0.631034
3,2014-01-07,9.8413,11.783155,0.122921,0.0296,0.5527,0.204580,0.215122,0.205344,0.208349
4,2014-01-08,9.9233,3.096440,0.888659,0.0301,0.5527,6.919143,6.903296,6.918827,6.913756


In [67]:
df.to_csv("modified_QUANTT_dataset.csv")